# **📓 03: Lanzador de la Interfaz Web con Streamlit**

Este notebook tiene una única y clara responsabilidad: lanzar la interfaz de usuario web interactiva para el proyecto VIREC. La aplicación reside en su propio directorio (`streamlit_app/`) y este notebook actúa como el centro de control para seleccionarla y ejecutarla en un entorno de Colab.

**Flujo de Trabajo:**
1.  **Instalación y Configuración:** Se instalan las librerías, se monta Google Drive y se localiza dinámicamente la carpeta del proyecto.
2.  **Selección de Modelo:** El script lee el registro central de experimentos (`registro_de_experimentos.csv`) y te permite seleccionar interactivamente qué modelo deseas cargar en la interfaz a través de un menú desplegable.
3.  **Lanzamiento:** El notebook **crea un archivo de configuración temporal (`session_config.json`)** que contiene la ruta al modelo seleccionado y sus parámetros. Luego, ejecuta la aplicación `streamlit_app/app.py`, la cual lee este archivo para configurarse. Finalmente, crea un túnel público con `ngrok` para hacer la aplicación accesible.

### **Configuración del Entorno y Conexión**

Esta celda inicial se encarga de toda la preparación necesaria. Realiza las siguientes tareas en orden:

1.  **Montar Google Drive** y **localizar** la carpeta raíz del proyecto.
2.  **Instalar las dependencias** desde el archivo `requirements.txt`.
3.  **Importar** todas las librerías y las variables de configuración desde `config.py`.
4.  **Definir** las rutas a los directorios de la aplicación.

In [1]:
# ====================================================================================
# @title PASO 1: SETUP, CONFIGURACIÓN Y CONEXIÓN
# ====================================================================================
import os
import sys

# --- 1. Montar Google Drive y buscar la ruta del proyecto ---
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Google Drive ya está montado.")

NOMBRE_CARPETA_ANCLA = 'Proyecto_VIREC'
RUTA_BASE_PROYECTO = None
print(f"\nBuscando la carpeta ancla '{NOMBRE_CARPETA_ANCLA}'...")
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if NOMBRE_CARPETA_ANCLA in dirs:
        RUTA_BASE_PROYECTO = os.path.join(root, NOMBRE_CARPETA_ANCLA)
        break
if not RUTA_BASE_PROYECTO: raise FileNotFoundError(f"❌ ERROR CRÍTICO: No se pudo encontrar la carpeta '{NOMBRE_CARPETA_ANCLA}'.")
print(f"✅ ¡Proyecto encontrado! La ruta base es: {RUTA_BASE_PROYECTO}")

# --- 2. Instalar Dependencias ---
ruta_requirements = os.path.join(RUTA_BASE_PROYECTO, 'requirements.txt')
if os.path.exists(ruta_requirements):
    print("\nInstalando dependencias...")
    !pip install -r "{ruta_requirements}" -q
    print("✅ Dependencias instaladas.")
else:
    raise FileNotFoundError(f"❌ ERROR: No se encontró 'requirements.txt'.")

# --- 3. Importar Librerías y Configuración ---
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from pyngrok import ngrok
import time

sys.path.append(RUTA_BASE_PROYECTO)
try:
    from config import NGROK_AUTH_TOKEN
    print("✅ Configuración local importada desde config.py.")
except ImportError:
    NGROK_AUTH_TOKEN = None
    print("❌ ERROR: No se encontró 'config.py' o NGROK_AUTH_TOKEN no está definido.")

# --- 4. Construcción de Rutas para la App ---
RUTA_APP = os.path.join(RUTA_BASE_PROYECTO, 'streamlit_app')
RUTA_REGISTRO_MODELOS = os.path.join(RUTA_BASE_PROYECTO, 'modelos_entrenados', 'registro_de_experimentos.csv')
print("✅ Rutas de la aplicación configuradas.")
print("\n✅ Entorno completamente configurado.")

Mounted at /content/drive

Buscando la carpeta ancla 'Proyecto_VIREC'...
✅ ¡Proyecto encontrado! La ruta base es: /content/drive/MyDrive/Colab Notebooks/TalentoTech/Proyecto_VIREC

Instalando dependencias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00
✅ Dependencias instaladas.
✅ Configuración local importada desde config.py.
✅ Rutas de la aplicación configuradas.

✅ Entorno completamente configurado.


### **Selección de Modelo a Desplegar**
Esta celda lee el `registro_de_experimentos.csv` para encontrar todos los modelos que has entrenado.

👇 **Por favor, selecciona del menú desplegable el experimento que deseas cargar en la interfaz web antes de continuar con la siguiente celda.**

In [2]:
# ====================================================================================
# @title PASO 2: SELECCIONAR MODELO DESDE EL REGISTRO
# ====================================================================================

modelos_info = {} # Diccionario para guardar {nombre_display: {ruta: ..., img_size: ...}}

if not os.path.exists(RUTA_REGISTRO_MODELOS):
    print(f"❌ ADVERTENCIA: No se encontró el archivo de registro en '{RUTA_REGISTRO_MODELOS}'.")
else:
    df_registro = pd.read_csv(RUTA_REGISTRO_MODELOS)
    if 'experiment_id' in df_registro.columns and 'ruta_modelo_guardado' in df_registro.columns:
        for index, row in df_registro.iterrows():
            # Creamos un nombre amigable para mostrar en el menú
            display_name = f"{row['experiment_id']} (val_acc: {row['best_val_accuracy']})"
            # Guardamos la información necesaria para el lanzamiento
            modelos_info[display_name] = {
                "ruta_completa": os.path.join(RUTA_BASE_PROYECTO, row['ruta_modelo_guardado']),
                "img_size": int(row['img_size'])
            }

        if not modelos_info:
             print("❌ ADVERTENCIA: El registro de experimentos está vacío.")
        else:
            dropdown_modelos = widgets.Dropdown(
                options=modelos_info.keys(),
                description='Elige un experimento:',
                style={'description_width': 'initial'}, layout={'width': 'max-content'}
            )
            display(dropdown_modelos)
    else:
        print("❌ ADVERTENCIA: El archivo de registro no tiene las columnas requeridas.")

Dropdown(description='Elige un experimento:', layout=Layout(width='max-content'), options=('Prueba_Parametros_…

### **Lanzamiento de la Aplicación Web**
Esta celda final toma tu selección del menú anterior y lanza la aplicación de Streamlit, creando un túnel público con ngrok para que puedas acceder a ella.

In [3]:
# ====================================================================================
# @title PASO 3: LANZAR LA APLICACIÓN WEB
# ====================================================================================
from pyngrok import ngrok
import time
import sys
import json

# --- Importar la configuración de NGROK ---
sys.path.append(RUTA_BASE_PROYECTO)
try:
    from config import NGROK_AUTH_TOKEN
except ImportError:
    NGROK_AUTH_TOKEN = None

if NGROK_AUTH_TOKEN and 'dropdown_modelos' in locals():
    # --- 1. Obtener los parámetros del modelo seleccionado ---
    seleccion = dropdown_modelos.value
    info_modelo = modelos_info[seleccion]
    ruta_modelo_a_lanzar = info_modelo['ruta_completa']
    img_size_a_lanzar = info_modelo['img_size']

    print(f"🚀 Preparando para lanzar el modelo: {os.path.basename(ruta_modelo_a_lanzar)}")

    # --- 2. Moverse al directorio de la app ---
    RUTA_APP = os.path.join(RUTA_BASE_PROYECTO, 'streamlit_app')
    %cd {RUTA_APP}

    # --- 3. CREAR EL ARCHIVO DE CONFIGURACIÓN DE SESIÓN ---
    # Este archivo le pasará los parámetros a la app de forma segura
    config_data = {
        "model_path": ruta_modelo_a_lanzar,
        "img_size": img_size_a_lanzar
    }
    with open('session_config.json', 'w') as f:
        json.dump(config_data, f)
    print("✅ Archivo de configuración de sesión 'session_config.json' creado.")

    # --- 4. Limpiar sesiones anteriores y lanzar la app (sin argumentos) ---
    os.system("killall streamlit")
    ngrok.kill()

    comando_lanzamiento = "nohup streamlit run app.py &"
    print(f"Ejecutando comando: {comando_lanzamiento}")
    get_ipython().system(comando_lanzamiento)

    time.sleep(5)

    # --- 5. Conectar ngrok ---
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(8501)
    print("\n" + "="*60)
    print("🎉 ¡Tu aplicación está en línea!")
    print(f"🔗 Abre esta URL en tu navegador: {public_url}")
    print("="*60)

    %cd /content
else:
    print("❌ ERROR: Asegúrate de ejecutar la celda anterior y de tener tu token de Ngrok en config.py.")

🚀 Preparando para lanzar el modelo: 2025-09-17_lr-0.001_bs-32_is-224_aug-T_gray-F_best.keras
/content/drive/MyDrive/Colab Notebooks/TalentoTech/Proyecto_VIREC/streamlit_app
✅ Archivo de configuración de sesión 'session_config.json' creado.
Ejecutando comando: nohup streamlit run app.py &
nohup: appending output to 'nohup.out'

🎉 ¡Tu aplicación está en línea!
🔗 Abre esta URL en tu navegador: NgrokTunnel: "https://cf8cde8044f9.ngrok-free.app" -> "http://localhost:8501"
/content
